In [ ]:
import os, sys
import scipy.misc
from glob import glob
import numpy as np
import random 
import shutil 
import keras
import matplotlib.pyplot as plt

base_path = os.path.abspath("../")
data_name = "cxr"


sys.path.append(base_path)
import helpers

In [ ]:
def load_img(path):
    # read image
    img = scipy.misc.imread(path, mode='RGB').astype(np.float)
    #  resize
    img = scipy.misc.imresize(img, (256,256))
    
    # rescale to [-1, 1]
    #img = img/127.5 - 1.
    # rescale to [0, 1]
    #img /= 255.0
    
    return img

In [ ]:
# Image paths
cov19 = os.path.join(base_path, "dataset/{}/train/cov19".format(data_name))
normal = os.path.join(base_path, "dataset/{}/train/normal".format(data_name))

g1 = os.path.join(base_path, "dataset/{}/train/from_normal".format(data_name))
g2 = os.path.join(base_path, "dataset/{}/train/from_pneu".format(data_name))

# Destinations

# Normal to COVID: n2c
dest_path = os.path.join(base_path, "dataset/numpy/")
helpers.create_directory(dest_path)

In [ ]:
cov19_path = [os.path.join(cov19, x) for x in os.listdir(cov19)]
normal_path = [os.path.join(normal, x) for x in os.listdir(normal)]
g1_path = [os.path.join(g1, x) for x in os.listdir(g1)]
g2_path = [os.path.join(g2, x) for x in os.listdir(g2)]

#pneu_path = [os.path.join(pneu, x) for x in os.listdir(pneu)]

len(cov19_path), len(normal_path), len(g1_path), len(g2_path)

In [ ]:
random.shuffle(cov19_path)
random.shuffle(normal_path)
random.shuffle(g1_path)
random.shuffle(g2_path)

cov19_path = cov19_path[:180]
normal_path = normal_path[:180]
g1_path = g1_path[:180]
g2_path = g2_path[:180]

In [ ]:
covid19_images=np.array([load_img(path) for path in cov19_path])
covid19_images.shape

In [ ]:
normal_images=np.array([load_img(path) for path in normal_path])
normal_images.shape

In [ ]:
g1_images=np.array([load_img(path) for path in g1_path])
g1_images.shape

In [ ]:
g2_images=np.array([load_img(path) for path in g2_path])
g2_images.shape

In [ ]:
#np.save("pneumonia.npy", normal_images)

In [ ]:
#normal_images = np.load("n.npy")

In [ ]:
#---------------------------
# cxr
# 0 - cov
# 1 - normal
# 2 - pneu
#---------------------------
# cxr_normalvcovid
# 0 - cov19
# 1 - normal
#---------------------------
# cxr_pneucovid
# 0 - cov19
# 1 - pneu
#---------------------------

In [ ]:
# Normal vs Covid
#normal_labels = np.array([1 for _ in range(len(normal_images))])
#cov19_labels = np.array([0 for _ in range(len(covid19_images))])

normal_labels = np.array([0 for _ in range(len(normal_images))])
cov19_labels = np.array([1 for _ in range(len(covid19_images))])
g1_labels = np.array([2 for _ in range(len(g1_images))])
g2_labels = np.array([3 for _ in range(len(g2_images))])

print(cov19_labels.shape, normal_labels.shape)

x_all = np.concatenate((normal_images, covid19_images, g1_images, g2_images), axis=0)
y_all = np.concatenate((normal_labels, cov19_labels, g1_labels, g2_labels), axis=0)
y_all = keras.utils.to_categorical(y_all, 4)

print(x_all.shape, y_all.shape)

# only test set
#np.save("{}/{}_x.npy".format(dest_path, data_name), x_all)
#np.save("{}/{}_y.npy".format(dest_path, data_name), y_all)

In [ ]:
#x_all = np.load("{}/{}_x.npy".format(dest_path, data_name))
#y_all = np.load("{}/{}_y.npy".format(dest_path, data_name))
#x_all.shape, y_all.shape

In [ ]:
from keras.models import load_model

EXPERIMENT_NAME =  "cxr_normalvcovid+g1g2" 
LOG_PATH = os.path.join(base_path, "models", EXPERIMENT_NAME)
model = None
model = load_model("{}/{}.h5".format(LOG_PATH, EXPERIMENT_NAME), compile = False)
model.summary()

In [ ]:
#from keras.applications.vgg16 import VGG16
#base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(256, 256, 3))

#model = None
#model = base_model
#model.summary()

In [ ]:
from keras.models import Model

layer_name = 'global_average_pooling2d_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

intermediate_output = intermediate_layer_model.predict(x_all, verbose=1)
intermediate_output.shape

In [ ]:
intermediate_output.shape, y_all.shape

In [ ]:
import umap
import time
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import seaborn as sns


np.random.seed(42)

sns.set(context="paper", style="white")

y3 = y_all
raw_train = intermediate_output #x3
raw_annot = y3


print(raw_train.shape)
raw_t_s = np.array([img.flatten() for img in raw_train])
print(raw_t_s.shape)
print(raw_annot.shape)
raw_annot_flat = np.argmax(raw_annot, axis=1)
print(raw_annot_flat.shape)

raw_annot_flat_3 = raw_annot_flat
print(np.unique(raw_annot_flat_3))
print(raw_t_s.shape, raw_annot_flat_3.shape)


data = raw_t_s

reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(data)

colour_map = raw_annot_flat_3

tsneFigure = plt.figure(figsize=(12, 10))
fig, ax = plt.subplots(figsize=(12, 10))

for colour in range(2): # 1 - benign only, 2- malig benign, 3 - malig benign synth malig
    indices = np.where(colour_map==colour)
    indices = indices[0]
    
    if colour == 0:
        l = "Normal"
    if colour == 1:
        l = "COVID-19"
    if colour == 2:
        l = "G_NC"
    if colour == 3:
        l = "G_PC"
        
    plt.setp(ax, xticks=[], yticks=[])
    plt.scatter(embedding[:, 0][indices],
                embedding[:, 1][indices],
                label=l, cmap="Spectral", s=50)
    

#plt.title("UMAP of feature space")
plt.legend(loc='center left', prop={'size': 15})
plt.axis('off')
plt.savefig("{}/{}_1.pdf".format(LOG_PATH, data_name), bbox_inches = 'tight', pad_inches = 0, dpi=100)
plt.show()